Predictor Training
==============

To maximize its accuracy, the yield predictor is periodically re-trained on encoder output during encoder training. However, it is beneficial to seed the predictor by training it on random sequences.

In [1]:
# TODO: Delete this whole file once comments are ported to primo-sim-search https://github.com/uwmisl/cas9-similarity-search/issues/10

# The following diagram illustrates what the predictor training does relative to the rest of the flow. 


![TrainPredictor](../../documentation/documentation_01_train_predictor)

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import cupyck
import primo.models
import primo.tools.sequences as seqtools
import primo.models.cas9 as cas9


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-nnhexiqn because the default path (/tf/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Initialize the cupyck session first, otherwise Tensorflow will use up all available GPU memory:

In [3]:

#cupyck_sess = cupyck.GPUSession(max_seqlen=200, nblocks=1024, nthreads=128)

In [4]:
simulator = primo.models.Simulator()

Generate a large set of pairs with various Hamming distances, then simulate their query-target yields with NUPACK:

In [5]:
# Generate two random DNA sequences. First by creating a random sequence, then copying that sequence to another sequence, then mutating that second sequence at some given mutation rate, such that 
# the hamming distance between the sequences (i.e. how different they are) is sampled from a uniform distribution. This means that we could expect sequence that are nearly identical, and sequence pairs that are entirely opposite to occur at approximately the same rate. 
# Random_pairs: (arbitrary_sequence_1, arbitrary_sequence_2)

# Generate 5,000 of these arbitrary sequence pairs.
n_sequence_pairs_to_generate = 50000

# ...of length 80 nucleotides each (80 was chosen because that's the length of the DNA sequence's feature region in callie original paper).
sequence_length = 20

random_pairs, mut_rates = seqtools.random_mutant_pairs(n_sequence_pairs_to_generate, sequence_length)

In [6]:
# From here, we take the first arbitrary sequence, and its mutation (i.e. the random pairs), then we pretend that the first arbitrary sequence (i.e. at index 0) is a target DNA sequence, then we pretend the second arbitrary sequence (i.e. at index 1) is a query DNA sequences. 
seq_pairs = pd.DataFrame({
    "target_features": random_pairs[:, 0],
    "query_features": random_pairs[:, 1]
})

In [7]:
# Use NUPACK/CUPYCK to simulate the hybdrization yields of the two sequences in the pair (this is the Pink flow in the above diagram). 

# Note that this will not be used for Cas9 similarity search.
# Q: Why?
# A: This process of training a differentiable yield predictor from yields calculated by a *non*-differentiable yield simulator (i.e. NUPACK) is done becauase the non-differentiable simualtor is non-differentiable-- and therefore can't be used in a neural network (it would fail the backpropagation step).

# However, in Cas9 similarity search, we are predicting how well Cas9 will cut a target DNA sequence # using differentiable predictor provided by Finklestein lab called Nucleaseq (https://github.com/finkelsteinlab/nucleaseq) 

sim_results = simulator.simulate(seq_pairs)

Initialize a random predictor and train it on the sequences and simulated yields:

In [8]:
predictor = primo.models.PredictorModel()

In [9]:
onehot_seq_pairs = predictor.seq_pairs_to_onehots(seq_pairs)

AttributeError: 'PredictorModel' object has no attribute 'seq_pairs_to_onehots'

In [ ]:
history = predictor.train(onehot_seq_pairs, sim_results, epochs=50, validation_split = 0.2)

Visualize the model's accuracy by plotting the simulated yield against the pairwise Hamming distance, and coloring by the difference between the predicted yield and the simulated yield. The predictor should be most accurate along an S-shaped region in the center.

In [ ]:
pred_yield = predictor.model.predict(onehot_seq_pairs)

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(mut_rates, sim_results, c = np.abs(pred_yield.flatten() - sim_results))
plt.xlabel("Hamming distance")
plt.ylabel("Simulated yield")
plt.colorbar(label="|Simulated - Predicted|")

In [ ]:
errors = np.abs(pred_yield.flatten() - sim_results)
plt.figure()
plt.hist(errors)

Save the predictor for future training and use:

In [ ]:
predictor.save('/tf/primo/data/models/yield-model.h5')